In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
movie = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [ ]:
print(movie.shape)
print(credits.shape)

In [ ]:
movie.isnull().sum()

In [ ]:
credits.isnull().sum()

In [ ]:
movie.duplicated().sum()

In [ ]:
credits.duplicated().sum()

In [ ]:
movie.dropna(subset=['overview'],inplace=True)

In [ ]:
movie.isnull().sum()

In [ ]:
movie.head(2)

In [ ]:
# title
# genres
# keywords
# overview
new_movie = movie[['id','title','genres','keywords','overview']]

In [ ]:
new_movie.head()

In [ ]:
credits.head()

In [ ]:
final = new_movie.merge(credits,left_on='id',right_on='movie_id')[['title_x','genres','keywords','overview','cast','crew']]

In [ ]:
final.shape

In [ ]:
final.head(2)

In [ ]:
final.rename(columns={'title_x':'title'},inplace=True)

In [ ]:
final.head()

In [ ]:
final.iloc[0].genres

In [ ]:
# ["Action","Adventure","Fantasy","Sci"]
for i in '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]':
    print(i)

In [ ]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
def clean_col(text):
    x = []
    for i in ast.literal_eval(text):
        x.append(i['name'])
    y = x[:]
    return y
        

In [ ]:
clean_col('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
final['genres'] = final['genres'].apply(clean_col)

In [ ]:
final.head()

In [ ]:
final['keywords'] = final['keywords'].apply(clean_col)

In [ ]:
final.head()

In [ ]:
final['overview'] = final['overview'].apply(lambda x:x.split())

In [ ]:
final.head()

In [ ]:
final['cast'][0]

In [ ]:
def extract_cast(text):
    x=[]
    counter = 0
    for i in ast.literal_eval(text):
        if counter == 3:
            break
        x.append(i['name'])
        counter+=1
    y = x[:]
    return y 

In [ ]:
final['cast'] = final['cast'].apply(extract_cast)

In [ ]:
final.head()

In [ ]:
final['crew'][0]

In [ ]:
def extract_crew(text):
    x=[]
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            x.append(i['name'])
    y = x[:]
    return y

In [ ]:
final['crew'] = final['crew'].apply(extract_crew)

In [ ]:
final.head()

In [ ]:
final['metadata'] = final['genres'] + final['keywords'] + final['overview'] + final['cast'] + final['crew']

In [ ]:
final.head()

In [ ]:
df = final[['title','metadata']]

In [ ]:
df['metadata'][0]

In [ ]:
def remove_spaces(L):
    x=[]
    for i in L:
        x.append(i.replace(" ",""))
    y = x[:]
    return y

In [ ]:
df['metadata'] = df['metadata'].apply(remove_spaces)

In [ ]:
df.head()

In [ ]:
df['metadata'] = df['metadata'].apply(lambda x:" ".join(x))

In [ ]:
df.head()

In [ ]:
df['metadata'] = df['metadata'].apply(lambda x:x.lower())

In [ ]:
df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(stop_words='english',max_features=5000)

In [ ]:
X = cv.fit_transform(df['metadata']).toarray()

In [ ]:
X.shape

In [ ]:
X[0]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_matrix = cosine_similarity(X)

In [ ]:
similarity_matrix[0]

In [ ]:
def recommend(movie):
    # recommend 5 closest recommendations
    movie_index = df[df['title'] == movie].index[0]
    similarity_scores = similarity_matrix[movie_index]
    values = sorted(list(enumerate(similarity_scores)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in values:
        print(df.iloc[i[0]].title)
        
    

In [ ]:
recommend("21 Jump Street")

In [ ]:
sorted(list(enumerate(similarity_matrix[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
df.head()

In [ ]:
df.iloc[539].title

In [ ]:
df.sample(5)